In [1]:
import os
import numpy as np
import DataPreprocessing as DP
from matplotlib import pyplot as plt
from astropy.io import fits
#from skimage import exposure
#------------------------------------------------------------------------#
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, Reshape, RNN
from keras.models import Model
from keras import callbacks, optimizers
import tensorflow as tf

import time
Tstart = time.time() #Timer start

In [2]:
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #for GPU usagegpus = tf.config.experimental.list_physical_devices(device_type='XLA_GPU')
gpus = tf.config.experimental.list_physical_devices(device_type='XLA_GPU')
print(gpus)
tf.config.experimental.set_visible_devices(devices=gpus, device_type='XLA_GPU')

[PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]


In [3]:
#norm spectrum
kinds = ['boss_cv','boss_da+ms','boss_db','boss_db+ms','boss_dq','boss_dz','fgkm','hotstars','wd','wdsb2','yso','hotstars_m']
flux_cv, spectrum_cv = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[0]+'/'+'*.fit')
flux_dams, spectrum_dams = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[1]+'/'+'*.fit')
flux_db, spectrum_db = DP.Preprocessing7('/home/njl/ML/optical/'+kinds[2]+'/'+'*.fit')

In [4]:
#delate bad data
n = 0
for i in range(len(spectrum_db)):
    if len(spectrum_db[i][0]) < 4096:
        n = i
flux_db.pop(n)
spectrum_db.pop(n)

[array([5282.0195, 5283.235 , 5284.453 , 5285.669 , 5286.8877, 5288.104 ,
        5289.323 , 5290.54  , 5291.7573, 5292.9775, 5294.1953, 5295.416 ,
        5296.6343, 5297.8555, 5299.074 , 5300.2935, 5301.5156, 5302.7354,
        5303.958 , 5305.178 , 5306.4014, 5307.622 , 5308.8433, 5310.0674,
        5311.289 , 5312.5137, 5313.736 , 5314.9614, 5316.184 , 5317.407 ,
        5318.6333, 5319.857 , 5321.0835, 5322.3076, 5323.5347, 5324.76  ,
        5325.9844, 5327.213 , 5328.438 , 5329.667 , 5330.893 , 5332.122 ,
        5333.349 , 5334.5757, 5335.806 , 5337.0337, 5338.264 , 5339.492 ,
        5340.7236, 5341.952 , 5343.181 , 5344.413 , 5345.6426, 5346.8755,
        5348.1055, 5349.339 , 5350.5693, 5351.8003, 5353.034 , 5354.2656,
        5355.5005, 5356.7324, 5357.968 , 5359.2   , 5360.433 , 5361.6694,
        5362.903 , 5364.139 , 5365.3735, 5366.611 , 5367.845 , 5369.08  ,
        5370.3184, 5371.5537, 5372.7925, 5374.0283, 5375.2676, 5376.504 ,
        5377.744 , 5378.981 , 5380.218

In [26]:
l_list =  flux_cv+flux_dams+flux_db
sl = []
for i in range(len(l_list)):
    sl.append(len(l_list[i]))
sl_min = np.min(sl)
X_train = []
for i in range(len(l_list)):
    X_train.append(l_list[i][0:sl_min])
size = 4552
X_train = np.array(X_train)

In [27]:
len(X_train)

1214

In [27]:
X_train = X_train.T

In [16]:
X_train = np.stack(X_train)
#X_train = X_train.reshape(1,len(X_train),size)
print(X_train.shape)
#print(X_train.shape)

(1214, 4552)


In [33]:
X_train = X_train.reshape(len(X_train),1,X_train.shape[1])

In [34]:
X_train.shape

(1214, 1, 4552)

In [37]:
#parameters for LSTM
nb_lstm_outputs = 128  #神经元个数
nb_time_steps = len(l_list) #时间序列长度
nb_input_vector = sl_min #输入序列

In [28]:
model = Sequential()
model.add(LSTM(32, input_dim=4552, input_length=1214, return_sequences=True))
# note that you only need to specify the input size on the first layer.
# for subsequent layers, no need to specify the input size:
model.add(LSTM(16, return_sequences=True))
model.add(LSTM(10))

In [29]:
optimizer_adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer_adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1214, 32)          586880    
_________________________________________________________________
lstm_7 (LSTM)                (None, 1214, 16)          3136      
_________________________________________________________________
lstm_8 (LSTM)                (None, 10)                1080      
Total params: 591,096
Trainable params: 591,096
Non-trainable params: 0
_________________________________________________________________


In [36]:
Nepochs = 100 #number of epochs for CAE training
tosavemodel = True #if save the trained CAE model
savename = 'LSTM_reconstruction' #setup if "tosavemodel=True" or "plot_reconstuction=True"

In [37]:
Tprocess0 = time.time()
print('\n', '## DATE PREPARATION RUNTIME:', Tprocess0-Tstart) #Timer

## MAIN ##
#training
H = model.fit(X_train, X_train,
                epochs=Nepochs,
                shuffle=True)

Tprocess1 = time.time()
print('\n', '## AE TRAINING RUNTIME:', Tprocess1-Tprocess0) #Timer


 ## DATE PREPARATION RUNTIME: 564.3310761451721
Epoch 1/100


ValueError: in user code:

    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    /home/njl/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/